In [1]:
!python -V

Python 3.10.12


In [2]:
import pandas as pd

In [3]:
import pickle

In [4]:
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [ ]:
import mlflow


mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

In [7]:
!wget -O ./data/green_tripdata_2021-01.parquet https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet
!wget -O ./data/green_tripdata_2021-02.parquet https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-02.parquet


--2023-07-04 16:51:26--  https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 2600:9000:2574:8a00:b:20a5:b140:21, 2600:9000:2574:f000:b:20a5:b140:21, 2600:9000:2574:9a00:b:20a5:b140:21, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|2600:9000:2574:8a00:b:20a5:b140:21|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1333519 (1.3M) [binary/octet-stream]
Saving to: ‘./data/green_tripdata_2021-01.parquet’

./data/green_tripda 100%[===================>]   1.27M  --.-KB/s    in 0.1s    

2023-07-04 16:51:26 (8.48 MB/s) - ‘./data/green_tripdata_2021-01.parquet’ saved [1333519/1333519]

--2023-07-04 16:51:26--  https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-02.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 2600:9000:2574:e800:b:20a5:b140:21, 2600:9000:2574:f000:b:20a5:b140:21, 2600

In [8]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [9]:
df_train = read_dataframe('./data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('./data/green_tripdata_2021-02.parquet')

In [10]:
len(df_train), len(df_val)

(73908, 61921)

In [11]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [12]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [13]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [14]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

7.758715210403367

In [15]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [26]:
with mlflow.start_run():

    mlflow.set_tag("developer", "dtn")

    mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.parquet")
    mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.parquet")

    alpha = 0.1
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

# Logging the Model
    mlflow.log_artifact(local_path="models/lin_reg.bin", artifact_path="models_pickle")

# XGBoost Model

In [17]:
import xgboost as xgb

In [18]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [19]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [20]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [21]:
# Hyperparameters search

search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

[16:57:47] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:20.00861                          
[1]	validation-rmse:18.91146                          
[2]	validation-rmse:17.89498                          
[3]	validation-rmse:16.95430                          
[4]	validation-rmse:16.08631                          
[5]	validation-rmse:15.28303                          
[6]	validation-rmse:14.54300                          
[7]	validation-rmse:13.85778                          
[8]	validation-rmse:13.22562                          
[9]	validation-rmse:12.64388                          
[10]	validation-rmse:12.11369                         
[11]	validation-rmse:11.62291                         
[12]	validation-rmse:11.17082                         
[13]	validation-rmse:10.75746                         
[14]	validation

KeyboardInterrupt: 

In [22]:
mlflow.xgboost.autolog(disable=True)

In [23]:
# Best parameters with preprocessor:

with mlflow.start_run():
    
    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)

    best_params = {
        'learning_rate': 0.11572976949838075,
        'max_depth': 62,
        'min_child_weight': 1.8139859191790475,
        'objective': 'reg:linear',
        'reg_alpha': 0.20653355807056745,
        'reg_lambda': 0.012740593248323861,
        'seed': 42
    }

    mlflow.log_params(best_params)

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )

    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
        
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

[17:13:19] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:19.12666
[1]	validation-rmse:17.32676
[2]	validation-rmse:15.75631
[3]	validation-rmse:14.39402
[4]	validation-rmse:13.21749
[5]	validation-rmse:12.20037
[6]	validation-rmse:11.32675
[7]	validation-rmse:10.58140
[8]	validation-rmse:9.94517
[9]	validation-rmse:9.40514
[10]	validation-rmse:8.94828
[11]	validation-rmse:8.56239
[12]	validation-rmse:8.23868
[13]	validation-rmse:7.96401
[14]	validation-rmse:7.73804
[15]	validation-rmse:7.54750
[16]	validation-rmse:7.38658
[17]	validation-rmse:7.24994
[18]	validation-rmse:7.13743
[19]	validation-rmse:7.04263
[20]	validation-rmse:6.96151
[21]	validation-rmse:6.89300
[22]	validation-rmse:6.83397
[23]	validation-rmse:6.78523
[24]	validation-rmse:6.74310
[25]	validation-rmse:6.70678
[26]	validation-rmse:6.67673
[27]

/Users/daniloneves/opt/anaconda3/envs/mlflow/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [25]:
# With autolog:

best_params = {
    'learning_rate': 0.11572976949838075,
    'max_depth': 62,
    'min_child_weight': 1.8139859191790475,
    'objective': 'reg:linear',
    'reg_alpha': 0.20653355807056745,
    'reg_lambda': 0.012740593248323861,
    'seed': 42
}

mlflow.xgboost.autolog()

booster = xgb.train(
    params=best_params,
    dtrain=train,
    num_boost_round=1000,
    evals=[(valid, 'validation')],
    early_stopping_rounds=50
)


2023/07/04 20:01:39 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of xgboost. If you encounter errors during autologging, try upgrading / downgrading xgboost to a supported version, or try upgrading MLflow.
2023/07/04 20:01:39 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '6a6e314fb37049f892cd14c58f1bf11b', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current xgboost workflow


[20:01:50] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:19.12666
[1]	validation-rmse:17.32676
[2]	validation-rmse:15.75631
[3]	validation-rmse:14.39402
[4]	validation-rmse:13.21749
[5]	validation-rmse:12.20037
[6]	validation-rmse:11.32675
[7]	validation-rmse:10.58140
[8]	validation-rmse:9.94517
[9]	validation-rmse:9.40514
[10]	validation-rmse:8.94828
[11]	validation-rmse:8.56239
[12]	validation-rmse:8.23868
[13]	validation-rmse:7.96401
[14]	validation-rmse:7.73804
[15]	validation-rmse:7.54750
[16]	validation-rmse:7.38658
[17]	validation-rmse:7.24994
[18]	validation-rmse:7.13743
[19]	validation-rmse:7.04263
[20]	validation-rmse:6.96151
[21]	validation-rmse:6.89300
[22]	validation-rmse:6.83397
[23]	validation-rmse:6.78523
[24]	validation-rmse:6.74310
[25]	validation-rmse:6.70678
[26]	validation-rmse:6.67673
[27]

2023/07/04 20:05:33 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.


In [23]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR

mlflow.sklearn.autolog()

for model_class in (RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, LinearSVR):

    with mlflow.start_run():

        mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.parquet")
        mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.parquet")
        mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

        mlmodel = model_class()
        mlmodel.fit(X_train, y_train)

        y_pred = mlmodel.predict(X_val)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)
        

/Users/cristian.martinez/miniconda3/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [32]:
# Best parameters without preprocessor:
with mlflow.start_run():

    best_params = {
        'learning_rate': 0.09585355369315604,
        'max_depth': 30,
        'min_child_weight': 1.060597050922164,
        'objective': 'reg:linear',
        'reg_alpha': 0.018060244040060163,
        'reg_lambda': 0.011658731377413597,
        'seed': 42
        }

    mlflow.log_params(best_params)

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )

    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

[20:22:29] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:19.48425
[1]	validation-rmse:17.95635
[2]	validation-rmse:16.59103
[3]	validation-rmse:15.37412
[4]	validation-rmse:14.29029
[5]	validation-rmse:13.32807
[6]	validation-rmse:12.47571
[7]	validation-rmse:11.72138
[8]	validation-rmse:11.05825
[9]	validation-rmse:10.47535
[10]	validation-rmse:9.96449
[11]	validation-rmse:9.51756
[12]	validation-rmse:9.12641
[13]	validation-rmse:8.78569
[14]	validation-rmse:8.49089
[15]	validation-rmse:8.23445
[16]	validation-rmse:8.01207
[17]	validation-rmse:7.81886
[18]	validation-rmse:7.65169
[19]	validation-rmse:7.50672
[20]	validation-rmse:7.38153
[21]	validation-rmse:7.27299
[22]	validation-rmse:7.17863
[23]	validation-rmse:7.09718
[24]	validation-rmse:7.02603
[25]	validation-rmse:6.96344
[26]	validation-rmse:6.90935
[2